In [1]:
#匯入所需模組

import os
import pandas as pd
import numpy as np
from keras.preprocessing import image
from keras.utils import np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.applications import InceptionV3
from keras.optimizers import Adam
from keras.layers import Dense, Activation, Flatten
from keras.layers import Input,Dropout
from keras.models import Model

Using TensorFlow backend.


In [2]:
#讀進訓練集並做正規化
path='./train_images/'
img_name=pd.read_csv("./train.csv")['ID']
img_data_list=[]

for img in img_name:
    img_path=path+img
    IMG=image.load_img(img_path,target_size=(224,224))
    x=image.img_to_array(IMG)
    x=np.expand_dims(x,axis=0)
    x=x/255.0
    img_data_list.append(x)

img_data = np.array(img_data_list)

print (img_data.shape)  
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape) 
img_data=img_data[0]
print (img_data.shape)  

(2528, 1, 224, 224, 3)
(1, 2528, 224, 224, 3)
(2528, 224, 224, 3)


In [3]:
#設定標籤值
train=pd.read_csv("./train.csv")
num_classes=6
labels=train['Label']
Y=np_utils.to_categorical(labels,num_classes)
x,y=shuffle(img_data,Y,random_state=2)

#訓練集80%、驗證集20%
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

In [4]:
#使用InceptionV3進行模型訓練
image_input=Input(shape=(224,224,3))
model=InceptionV3(include_top=False, weights='imagenet', input_shape=(224,224,3),input_tensor=image_input)

last_layer = model.output
last_layer=Flatten()(last_layer)
last_layer=Dropout(0.5)(last_layer)
out = Dense(num_classes, activation='softmax', name='softmax')(last_layer)
custom_V3_model = Model(image_input, out)
custom_V3_model.summary()

custom_V3_model.compile(loss='categorical_crossentropy',optimizer=Adam(lr=1e-5),metrics=['accuracy'])
custom_V3_model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=8,epochs=60,verbose=1)

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 111, 111, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 111, 111, 32) 0           batch_normalization_1[0][0]      
____________________

Instructions for updating:
Use tf.cast instead.
Train on 2022 samples, validate on 506 samples
Epoch 1/60
2022/2022 [==============================] - 98s 49ms/step - loss: 1.4019 - accuracy: 0.4862 - val_loss: 1.0333 - val_accuracy: 0.6838
Epoch 2/60
2022/2022 [==============================] - 78s 38ms/step - loss: 0.7216 - accuracy: 0.7517 - val_loss: 0.4604 - val_accuracy: 0.8597
Epoch 3/60
2022/2022 [==============================] - 78s 38ms/step - loss: 0.5219 - accuracy: 0.8229 - val_loss: 0.2843 - val_accuracy: 0.9190
Epoch 4/60
2022/2022 [==============================] - 78s 38ms/step - loss: 0.4149 - accuracy: 0.8645 - val_loss: 0.2486 - val_accuracy: 0.9269
Epoch 5/60
2022/2022 [==============================] - 78s 38ms/step - loss: 0.3195 - accuracy: 0.8858 - val_loss: 0.1580 - val_accuracy: 0.9447
Epoch 6/60
2022/2022 [==============================] - 78s 39ms/step - loss: 0.2648 - accuracy: 0.9120 - val_loss: 0.1445 - val_accuracy: 0.9565
Epoch 7/60
2022/2022 [=======

Epoch 55/60
2022/2022 [==============================] - 79s 39ms/step - loss: 0.0090 - accuracy: 0.9975 - val_loss: 0.0409 - val_accuracy: 0.9921
Epoch 56/60
2022/2022 [==============================] - 79s 39ms/step - loss: 0.0111 - accuracy: 0.9960 - val_loss: 0.0273 - val_accuracy: 0.9921
Epoch 57/60
2022/2022 [==============================] - 79s 39ms/step - loss: 0.0029 - accuracy: 0.9990 - val_loss: 0.0277 - val_accuracy: 0.9941
Epoch 58/60
2022/2022 [==============================] - 79s 39ms/step - loss: 0.0051 - accuracy: 0.9990 - val_loss: 0.0292 - val_accuracy: 0.9941
Epoch 59/60
2022/2022 [==============================] - 79s 39ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0331 - val_accuracy: 0.9921
Epoch 60/60
2022/2022 [==============================] - 79s 39ms/step - loss: 0.0054 - accuracy: 0.9990 - val_loss: 0.0463 - val_accuracy: 0.9822


In [5]:
#讀進測試集並做正規化
test_data_path=pd.read_csv('./test.csv')["ID"]
test_img_list=[]
for img in test_data_path:
    test_IMG=image.load_img('./test_images/'+img,target_size=(224,224))
    x=image.img_to_array(test_IMG)
    x=np.expand_dims(x,axis=0)
    x=x/255.0
    test_img_list.append(x)

test_img_data = np.array(test_img_list)
  
print (test_img_data.shape)  
test_img_data=np.rollaxis(test_img_data,1,0)
print (test_img_data.shape)  
test_img_data=test_img_data[0]
print (test_img_data.shape)  

(10142, 1, 224, 224, 3)
(1, 10142, 224, 224, 3)
(10142, 224, 224, 3)


In [6]:
#預測測試集
prediction=custom_V3_model.predict(test_img_data)
predict=np.argmax(prediction,axis=1)
sub=pd.read_csv('./test.csv')
sub['Label']=predict
sub.head()

,ID,Label
0,test_00000.png,1
1,test_00001.png,2
2,test_00002.png,5
3,test_00003.png,0
4,test_00004.png,2
